# Querying Azure Sentinel via REST API using a Personal Access Token (PAT)

## Get Workspaces in Subscription

### Configure variables. Change this based on your tenant, domain, and app registration details

In [9]:
<#
		`$config = @{
			tenantId = $tenantID                # This is the tenant ID
			appId = $appId                      # This is the client ID when you register your app
			appSecret = $appSecret              # This is the generated client secret in the client app
            sentinelWorkspaceID = "afadfafafaf" # Log analytics workspace sentinel is using
		}
<#

## Import credential from storage path or Export a new XML file using above filled in parameters

In [2]:
## Checks to see if config file exist and if not export credential file from above.
$storagePath = ".\AzSentinel.credential"
$config = (Import-CliXml -Path $storagePath)
if (!$config) {
    $config | Export-CliXml -Path $storagePath
} else {
    Write-Host -Foregroundcolor green "`nCredential file loaded from $($storagePath)"
}


Credential file loaded from .\AzSentinel.credential


## Authenticate with the MS Graph API via Personal Access Token (PAT)

In [4]:
Import-Module "..\..\Graph-API\modules\GraphAuthentication.psm1" -Force  ## Imports the module
#Get-Command -Module "GraphAuthentication"                  ## Shows what commands were imported

$resourceURI = "https://api.loganalytics.io"

## authenticate with the graph API via REST API
$aadToken = Invoke-GraphAuthenticatePAT -authvariables $config -resource $resourceURI
Write-Host ($aadToken -replace "\d","*")


----------------------------------------------------------------------------
Authentiating with Microsoft Graph API using a Personal Access Token (PAT)
https://docs.microsoft.com/en-us/azure/storage/common/storage-auth-aad-app
----------------------------------------------------------------------------
Authentication variables detected

Returning your Access Token. Be sure to insert it into a variable...

[Example] 
$AccessToken = Authenticate-Graph-PAT $config

eyJ*eXAiOiJKV*QiLCJhbGciOiJSUzI*NiIsIng*dCI*IlNzWnNCTmhaY*YzUTlTNHRycFFCVEJ*TlJSSSIsImtpZCI*IlNzWnNCTmhaY*YzUTlTNHRycFFCVEJ*TlJSSSJ*.eyJhdWQiOiJodHRwczovL*FwaS*sb*dhbmFseXRpY*MuaW*iLCJpc*MiOiJodHRwczovL*N*cy**aW*kb*dzLm*ldC*kZTQwY*Y*ZS*hZDVmLTQyNDUtYTMxNy*xNGJlMzljYmIwZWYvIiwiaWF*IjoxNTkyMzczNjAzLCJuYmYiOjE*OTIzNzM*MDMsImV*cCI*MTU*MjM*NzUwMywiYWlvIjoiNDJkZ*lKZzg*OUsycDFxdjVYd*ZIZXhubFZzM*FnQT*iLCJhcHBpZCI*ImFlNGZhODcwLTIzYWYtNGFlNS*hNmVkLTVhYjE*MTE*NThjNyIsImFwcGlkYWNyIjoiMSIsImlkcCI*Imh*dHBzOi*vc*RzLndpbmRvd*MubmV*L*RlNDBjZjd

### Put your Azure Sentinel Hunting Query here

In [5]:
## Enter your query here
$query = @"

SecurityAlert

"@

$timeSpan = "" ## You can leave blank or specify a date. But something HAS to be there. 
 
## This is the query converted to JSON format
$body = (@{
   query = $Query
   timespan = $timeSpan  
} | ConvertTO-JSON)
Write-Host -ForeGround Yellow "`nQuery"; $Query
Write-Host -ForeGround Yellow "JSON"; $body


Query

SecurityAlert

JSON
{
  "query": "\nSecurityAlert\n",
  "timespan": ""
}


## We will query the API

In [6]:
$URI = "$($resourceURI)/v1/workspaces/$($config.sentinelWorkspaceID)/query"
$header = @{ 
    'Content-Type' = 'application/json'
    Accept = 'application/json'
    Authorization = "Bearer $($aadToken)" 
}

Write-Host "URI: $($URI -replace "[0-9]","*")"
$header

URI: https://api.loganalytics.io/v*/workspaces/a*******-*bde-****-*f**-*c**a**cc***/query

Name                           Value
----                           -----
Authorization                  Bearer eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IlNzWnNCTmhaY0Yz…
Accept                         application/json
Content-Type                   application/json



In [7]:
#query Graph API and insert into $graphresponse variable
$Result = (Invoke-RestMethod -Method POST -Uri $URI -Headers $header -body $body -ErrorAction Stop)
$Result.tables.rows | Select -First 1

a8960181-0bde-4955-9f94-2c12a36cc900

Monday, May 25, 2020 7:42:54 PM
DLP Policy - Sharepoint - Sensitive files shared externally
DLP Policy - Sharepoint - Sensitive files shared externally
Medium
Activity policy 'DLP Policy - Sharepoint - Sensitive files shared externally' was triggered by 'Bob Smith (bob@jingtoso.com)'
MCAS
Microsoft
073A5B74-5D2B-3E70-BBCE-B1CE0EE5EC2E
dee5f39b-e1fe-ad36-dc38-aec97082cf8e


MCAS_ALERT_CABINET_EVENT_MATCH_AUDIT
Unknown

False
Monday, May 25, 2020 7:42:49 PM
Monday, May 25, 2020 7:42:49 PM
Monday, May 25, 2020 7:42:54 PM

{
  "Cloud Applications": "Microsoft SharePoint Online"
}
[
  {
    "$id": "3",
    "Name": "bob",
    "UPNSuffix": "jingtoso.com",
    "AadUserId": "6836ed6d-2a15-4889-ad24-fbda46b5ffda",
    "Type": "account"
  },
  {
    "$id": "4",
    "AppId": 20892,
    "Name": "Microsoft SharePoint Online",
    "InstanceName": "Microsoft SharePoint Online",
    "Type": "cloud-application"
  }
]
Detection


[
  {
    "Href": "https://skyserver.